In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2

import PIL.Image

print(tf.__version__)

In [ ]:
# Data preprocessing
# Please make sure the data has been downloaded and put in the right path.
# Check the README.md file on the project root directory for more instructions.

train_dir = 'data/train'
val_dir = 'data/test'
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(48,48),
        batch_size=128,
        # color_mode="gray_framescale",
        color_mode="grayscale",
        class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(48,48),
        batch_size=128,
        # color_mode="gray_framescale",
        color_mode="grayscale",
        class_mode='categorical')

In [ ]:
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

In [ ]:
# Train the model 
#Unless you're changing parameters and retraining a new model,
#you should skip this block if you've already have a trained model weights

emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])
emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 32,
        epochs=1,
        validation_data=validation_generator,
        validation_steps=7178 // 32)

In [ ]:
# Save the trained model weights so that it can be easily re-used without having to retrain the data
emotion_model.save_weights('emotion_model_weights.h5')

In [ ]:
# Load the previously saved weights
emotion_model.load_weights('emotion_model_weights.h5')

In [ ]:
# Check its accuracy

import sklearn
from sklearn.metrics import classification_report, confusion_matrix


batch_size = 128
img_rows, img_cols = 48, 48
nb_train_samples = 28709
nb_validation_samples = 7178 

class_labels = validation_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())

#Confution Matrix and Classification Report
Y_pred = emotion_model.predict_generator(validation_generator, nb_validation_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)

print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = list(class_labels.values())
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

plt.figure(figsize=(8,8))
cnf_matrix = confusion_matrix(validation_generator.classes, y_pred)

plt.imshow(cnf_matrix, interpolation='nearest')
plt.colorbar()
tick_marks = np.arange(len(classes))
_ = plt.xticks(tick_marks, classes, rotation=90)
_ = plt.yticks(tick_marks, classes)

# Test on a single Image

In [ ]:
# Replace cv2.show with the below function because cv2.show crashes

from IPython import display
from PIL import Image

def cv2_imshow(a):
    """A replacement for cv2.imshow() for use in Jupyter notebooks.
    Args:
    a : np.ndarray. shape (N, M) or (N, M, 1) is an NxM grayscale image. shape
      (N, M, 3) is an NxM BGR color image. shape (N, M, 4) is an NxM BGRA color
      image.
    """
    a = a.clip(0, 255).astype('uint8')
    # cv2 stores colors as BGR; convert to RGB
    if a.ndim == 3:
        if a.shape[2] == 4:
            a = cv2.cvtColor(a, cv2.COLOR_BGRA2RGBA)
        else:
            a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
    display.display(Image.fromarray(a))

In [ ]:
from os import listdir
from os.path import isfile, join
from keras.preprocessing.image import img_to_array

face_classifier = cv2.CascadeClassifier('./Haarcascades/haarcascade_frontalface_default.xml')
# face_classifier = cv2.CascadeClassifier('./Haarcascades/haarcascade_frontalface_alt2.xml')

def face_detector(img):
    # Convert image to grayscale
    gray = cv2.cvtColor(img.copy(),cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return (0,0,0,0), np.zeros((48,48), np.uint8), img
    
    allfaces = []   
    rects = []
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (48, 48), interpolation = cv2.INTER_AREA)
        allfaces.append(roi_gray)
        rects.append((x,w,y,h))
    return rects, allfaces, img

img = cv2.imread("data/validate/angry.jpg")  # provide path to an image
rects, faces, image = face_detector(img)
i = 0
for face in faces:
    roi = face.astype("float") / 255.0
    if np.sum([face]) != 0.0:
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)

        # make a prediction on the ROI, then lookup the class
        preds = emotion_model.predict(roi)[0]
        label = class_labels[preds.argmax()]   

        #Overlay our detected emotion on our pic
        label_position = (rects[i][0] + int((rects[i][1]/2)), abs(rects[i][2] - 10))
        i =+ 1
        cv2.putText(image, label, label_position , cv2.FONT_HERSHEY_SIMPLEX,1, (0,255,0), 2)
    else:
        cv2.putText(image, "No face found!", (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,255,0), 3)

# Show the image inline
cv2_imshow(image)

# Let's test on some of validation images

In [ ]:
from keras.preprocessing import image
from os import listdir
from os.path import isfile, join
#import re

def draw_test(name, pred, im, true_label):
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 160, 0, 0, 300 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, "predited - "+ pred, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.putText(expanded_image, "true - "+ true_label, (20, 120) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,255,0), 2)
    cv2.imshow(name, expanded_image)


def getRandomImage(path, img_width, img_height):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    final_path = file_path + "/" + image_name
    return image.load_img(final_path, target_size = (img_width, img_height),grayscale=True), final_path, path_class

# dimensions of our images
img_width, img_height = 48, 48

files = []
predictions = []
true_labels = []

# predicting random images. You can change the range value
for i in range(0, 5):
    path = './data/test/' 
    img, final_path, true_label = getRandomImage(path, img_width, img_height)
    files.append(final_path)
    true_labels.append(true_label)
    x = image.img_to_array(img)
    x = x * 1./255
    x = np.expand_dims(x, axis=0)
    images = np.vstack([x])
    classes = emotion_model.predict_classes(images, batch_size = 10)
    predictions.append(classes)
    
for i in range(0, len(files)):
    image = cv2.imread((files[i]))
    image = cv2.resize(image, None, fx=3, fy=3, interpolation = cv2.INTER_CUBIC)
    #draw_test("Prediction", class_labels[predictions[i][0]], image, true_labels[i])
    print("Prediction: " + class_labels[predictions[i][0]])
    print("True Label: " + true_labels[i])
    cv2_imshow(image)
    # cv2.waitKey(0)

# cv2.destroyAllWindows()

In [ ]:
# See model details
emotion_model.summary()

In [ ]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
emotion_model.save('saved_model/my_model')

In [ ]:
# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
emotion_model.save('emotion_model.h5')

# Convert to Tensorflow-lite

In [ ]:
#Unfortunately TFLiteConverter only works with the nightly build at the moment,
#therefore we have to install that

!pip install tf-nightly

In [ ]:
import tensorflow as tf
print(tf.__version__)
print(help(tf.lite.TFLiteConverter))

In [ ]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model('./saved_model/my_model') # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('emotion_model.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
# Another way - Load model using the .h5 model

from tensorflow.keras.models import load_model
emotion_model = load_model('emotion_model.h5')


# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(emotion_model)
tflite_model = converter.convert()

# Save the model.
with open('emotion_model_h5.tflite', 'wb') as f:
    f.write(tflite_model)